<a href="https://colab.research.google.com/github/heinohen/tko_7095_i2hlt/blob/main/week5_ex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install datasets more-itertools

In [6]:
import datasets
import sklearn.feature_extraction

In [7]:
data = datasets.load_dataset("imdb")

In [8]:
cvec = sklearn.feature_extraction.text.CountVectorizer(lowercase = False, stop_words = None, token_pattern=r"(?u)\b\w+\b" )
analyzer = cvec.build_analyzer()
analyzer('I have a dog at home, it likes to shred newspapers.')

['I',
 'have',
 'a',
 'dog',
 'at',
 'home',
 'it',
 'likes',
 'to',
 'shred',
 'newspapers']

## TASK A

In [9]:
# Tokenize the IMDB dataset

def tokenize(ex) -> dict:
  return {"tokenized":analyzer(ex["text"])}

data = data.map(tokenize, num_proc = 4)

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
from collections import Counter
from more_itertools import sliding_window
import tqdm

def generate_ngrams(dset, n ):
  for ex in tqdm.tqdm(dset):
    tokens = ["<bos>"] * (n-1) + ex["tokenized"]+["<eos>"]
    for ngram in sliding_window(tokens,n):
      yield ngram

## TASK B

In [11]:
# Concat all invid datasets (train, test, unlabelled) in IMDB
# The "master" dataset is a dict of thses, so dset.values() has the datasets of the individual sections
combined_dataset = datasets.concatenate_datasets(list(data.values()))

In [12]:
ngrams = {}

for ngram in generate_ngrams(combined_dataset, 4):
  ngrams[ngram] = 1

100%|██████████| 100000/100000 [01:18<00:00, 1271.29it/s]


In [13]:
ngrams

{('<bos>', '<bos>', '<bos>', 'I'): 1,
 ('<bos>', '<bos>', 'I', 'rented'): 1,
 ('<bos>', 'I', 'rented', 'I'): 1,
 ('I', 'rented', 'I', 'AM'): 1,
 ('rented', 'I', 'AM', 'CURIOUS'): 1,
 ('I', 'AM', 'CURIOUS', 'YELLOW'): 1,
 ('AM', 'CURIOUS', 'YELLOW', 'from'): 1,
 ('CURIOUS', 'YELLOW', 'from', 'my'): 1,
 ('YELLOW', 'from', 'my', 'video'): 1,
 ('from', 'my', 'video', 'store'): 1,
 ('my', 'video', 'store', 'because'): 1,
 ('video', 'store', 'because', 'of'): 1,
 ('store', 'because', 'of', 'all'): 1,
 ('because', 'of', 'all', 'the'): 1,
 ('of', 'all', 'the', 'controversy'): 1,
 ('all', 'the', 'controversy', 'that'): 1,
 ('the', 'controversy', 'that', 'surrounded'): 1,
 ('controversy', 'that', 'surrounded', 'it'): 1,
 ('that', 'surrounded', 'it', 'when'): 1,
 ('surrounded', 'it', 'when', 'it'): 1,
 ('it', 'when', 'it', 'was'): 1,
 ('when', 'it', 'was', 'first'): 1,
 ('it', 'was', 'first', 'released'): 1,
 ('was', 'first', 'released', 'in'): 1,
 ('first', 'released', 'in', '1967'): 1,
 ('relea

## TASK C

In [16]:
import numpy as np

def softmax(x):
  return np.exp(x) / sum(np.exp(x))

def sample_from(counts, temperature=1.0):
  """
  counts: List of counts that form the distribution
  temperature: The "how wild the generation should be" parameter, numbers close to 0 are very conservative,
  numbers close or above 1 lead to quite wild generations
  """

  counts_array = np.array(counts)

  # Make thse sum up to 1
  counts_array_norm = counts_array / counts_array.sum()

  # Divide by temperature, that is what the algorithm does
  counts_array_norm /= temperature

  # Renormalize into a distribution using the softmax function, that is what the algorithm does
  final_distribution = softmax(counts_array_norm)

  # A good way to sample from a distribution is the following function from numpy
  x = np.random.multinomial(n = 1, pvals = final_distribution)
  selected_word = np.argmax(x).flatten()
  return selected_word[0]

sample_from([1,1,1,17], temperature = 0.5)


3

## TASK D

In [17]:
from pprint import pprint

def generate(ngrams, n, max_len = 40, temperature = 1.0, prompt = None):
  """
  ngrams: the master dict
  max_len: how many words at maximum
  temperature: the generation temperature
  prompt: the initial prompt, as a tuple, if not given n-1 <bos> symbols will be used
  """

  if prompt is None:
    prompt = ["<bos>"] * (n-1)

  generated = list(prompt) # This list will grow with the words inserted

  for _ in range(max_len):
    ### CODE GOES HERE

    if generated[-1] == "<eos>": # Stop on end of sequence
      break

  return generated



